In [1]:
# Mount Google drive to upload datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# The path to the data on my drive
D = '/content/drive/My Drive/W266_Project_Data/pmi_data'

In [3]:
import pandas as pd
import numpy as np

In [4]:
# Load the eval files for each model's translations
bart_eval = pd.read_csv(D+"/predicted_text/siamese_evaluations_of_translations/bart_siamese_paraphrase_eval.csv")
bart_eval.drop(columns={'Unnamed: 0'}, inplace=True)
indictrans_eval = pd.read_csv(D+"/predicted_text/siamese_evaluations_of_translations/indictrans_siamese_paraphrase_eval.csv")
indictrans_eval.drop(columns={'Unnamed: 0'}, inplace=True)

In [5]:
# View the first few records in each file
bart_eval.head()

,language_task,target_sentence,bart_translation,probability_of_paraphrase,cosine_similarity_scores
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि बाबासाहब अम्बेडकर की कर...,0.992026,0.988444
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,व्याख्या करते हुए इस समारोह को आज बीजापुर में ...,0.990891,0.981364
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,0.990394,0.985965
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.990999,1.000000
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...",0.986538,0.997599


In [6]:
indictrans_eval.head()

,language_task,target_sentence,indictrans_translation,probability_of_paraphrase,cosine_similarity_scores
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,0.989960,0.948379
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व ...,0.989784,0.973164
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,0.990394,0.985965
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.990179,0.986373
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीबो...",0.987861,0.992806


In [7]:
result = pd.merge(bart_eval, indictrans_eval, how="outer", on=["target_sentence"])

In [8]:
# View several rows
result

,language_task_x,target_sentence,bart_translation,probability_of_paraphrase_x,cosine_similarity_scores_x,language_task_y,indictrans_translation,probability_of_paraphrase_y,cosine_similarity_scores_y
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि बाबासाहब अम्बेडकर की कर...,0.992026,0.988444,translate English to Hindi,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,0.989960,0.948379
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,व्याख्या करते हुए इस समारोह को आज बीजापुर में ...,0.990891,0.981364,translate English to Hindi,आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व ...,0.989784,0.973164
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,0.990394,0.985965,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,0.990394,0.985965
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.990999,1.000000,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.990179,0.986373
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...",0.986538,0.997599,translate English to Hindi,"इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीबो...",0.987861,0.992806
...,...,...,...,...,...,...,...,...,...
2995,translate English to Tamil,6. மியான்மர் மத்திய தேர்தல் ஆணையம்,6. மியான்மர் மத்திய தேர்தல் ஆணையம்,0.972689,1.000000,translate English to Tamil,மியான்மர் ஒன்றிய தேர்தல் ஆணையம் மற்றும்,0.248196,0.760211
2996,translate English to Tamil,“சுதந்திரம் பெற்ற பிறகு ஏராளமானவை நிறைவேற்றப்ப...,“சுதந்திரம் பெற்ற பிறகு ஏராளமானவை நிறைவேற்றப்ப...,0.981514,0.989659,translate English to Tamil,நாடு சுதந்திரம் அடைந்ததிலிருந்து பல சாதனைகள் ப...,0.633609,0.886990
2997,translate English to Tamil,"பொது மக்களின் பங்களிப்பு, தகவல் ஓட்டத்தின் முற...","பொது மக்களின் பங்களிப்பு, தகவல் ஓட்ட முறை, வளங...",0.947091,0.980610,translate English to Tamil,"மக்கள் பங்கேற்பு, தகவல் பரிமாற்றம், ஆதாரங்களை ...",0.338175,0.961721
2998,translate English to Tamil,"ஊரக சுயாட்சி திட்டம், வளமான பாரதம் போன்ற சமீபத...","ஊரக சுயாட்சி திட்டம்,வளமான இந்தியா போன்ற சமீபத...",0.258094,0.977657,translate English to Tamil,அண்மையில் மேற்கொள்ளப்பட்ட கிராம சுயாட்சி இயக்க...,0.090655,0.935723


In [9]:
# The language task field is also the same everywhere - we'll drop one instance of this column and rename the other
result['language_task_x'].equals(result['language_task_y'])

True

In [10]:
result.columns

Index(['language_task_x', 'target_sentence', 'bart_translation',
       'probability_of_paraphrase_x', 'cosine_similarity_scores_x',
       'language_task_y', 'indictrans_translation',
       'probability_of_paraphrase_y', 'cosine_similarity_scores_y'],
      dtype='object')

In [11]:
# Drop the redundant column
result.drop(columns={'language_task_y'}, inplace=True)

# Rename the columns of interest
result.rename(columns={'language_task_x': 'language_task', 'probability_of_paraphrase_x': 'bart_paraphrase_score',
                       'cosine_similarity_scores_x': 'bart_cosine_score', 'probability_of_paraphrase_y': 'indictrans_paraphrase_score',
                       'cosine_similarity_scores_y': 'indictrans_cosine_score'}, inplace=True)

In [12]:
result.head()

,language_task,target_sentence,bart_translation,bart_paraphrase_score,bart_cosine_score,indictrans_translation,indictrans_paraphrase_score,indictrans_cosine_score
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि बाबासाहब अम्बेडकर की कर...,0.992026,0.988444,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,0.989960,0.948379
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,व्याख्या करते हुए इस समारोह को आज बीजापुर में ...,0.990891,0.981364,आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व ...,0.989784,0.973164
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,0.990394,0.985965,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,0.990394,0.985965
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.990999,1.000000,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.990179,0.986373
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...",0.986538,0.997599,"इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीबो...",0.987861,0.992806


In [13]:
result.tail()

,language_task,target_sentence,bart_translation,bart_paraphrase_score,bart_cosine_score,indictrans_translation,indictrans_paraphrase_score,indictrans_cosine_score
2995,translate English to Tamil,6. மியான்மர் மத்திய தேர்தல் ஆணையம்,6. மியான்மர் மத்திய தேர்தல் ஆணையம்,0.972689,1.000000,மியான்மர் ஒன்றிய தேர்தல் ஆணையம் மற்றும்,0.248196,0.760211
2996,translate English to Tamil,“சுதந்திரம் பெற்ற பிறகு ஏராளமானவை நிறைவேற்றப்ப...,“சுதந்திரம் பெற்ற பிறகு ஏராளமானவை நிறைவேற்றப்ப...,0.981514,0.989659,நாடு சுதந்திரம் அடைந்ததிலிருந்து பல சாதனைகள் ப...,0.633609,0.886990
2997,translate English to Tamil,"பொது மக்களின் பங்களிப்பு, தகவல் ஓட்டத்தின் முற...","பொது மக்களின் பங்களிப்பு, தகவல் ஓட்ட முறை, வளங...",0.947091,0.980610,"மக்கள் பங்கேற்பு, தகவல் பரிமாற்றம், ஆதாரங்களை ...",0.338175,0.961721
2998,translate English to Tamil,"ஊரக சுயாட்சி திட்டம், வளமான பாரதம் போன்ற சமீபத...","ஊரக சுயாட்சி திட்டம்,வளமான இந்தியா போன்ற சமீபத...",0.258094,0.977657,அண்மையில் மேற்கொள்ளப்பட்ட கிராம சுயாட்சி இயக்க...,0.090655,0.935723
2999,translate English to Tamil,இந்தியாவின் 130 லட்சம் மக்கள் சார்பில் உங்கள் ...,130 லட்சம் இந்தியர்களுக்காக புதுதில்லிக்கு உங்...,0.145683,0.933602,இந்தியாவின் 130 கோடி மக்களின் சார்பாக உங்கள் அ...,0.713379,0.979156


In [15]:
# Check if any of the paraphrase scores are exactly equal
len(result[result.bart_paraphrase_score == result.indictrans_paraphrase_score])

25

In [16]:
# Check these 25 translations
same_question = result[result.bart_paraphrase_score == result.indictrans_paraphrase_score]
# Check if the translated texts are exactly the same - the translated sentences in these sentences are the same
same_question['bart_translation'].equals(same_question['indictrans_translation'])

True

In [17]:
# Create a new column using the "best" translation as measured by paraphrase score
# For equal sentences (IndicTrans and MBart produced the same output, we'll keep the Bart record)
result['best_trans'] = np.where(result.indictrans_paraphrase_score > result.bart_paraphrase_score, result.indictrans_translation, result.bart_translation)

In [18]:
# Number of times that Bart produced a better translation than Indictrans
len(result[result.best_trans == result.bart_translation]) - 25

2320

In [19]:
# Number of times that Indictrans produced a better translation than Bart
len(result[result.best_trans == result.indictrans_translation]) - 25

655

In [20]:
# Filter the dataset into its respective languages and save the target and results to compute language level sacre bleu
hi_result = result[result.language_task == 'translate English to Hindi']
ta_result = result[result.language_task == 'translate English to Tamil']
ml_result = result[result.language_task == 'translate English to Malayalam']

In [21]:
# Find the number of times that the 2 translators produced the same quality translation for each language
# 20 times the Hindi translations were of equal quality
print(len(hi_result[hi_result.bart_paraphrase_score == hi_result.indictrans_paraphrase_score]))
# 5 times the Tamil translations were of equal quality
print(len(ta_result[ta_result.bart_paraphrase_score == ta_result.indictrans_paraphrase_score]))
# 0 times the Malayalam translations were of equal quality
print(len(ml_result[ml_result.bart_paraphrase_score == ml_result.indictrans_paraphrase_score]))

20
5
0


In [22]:
# Find the number of times that MBart produced a better translation for Hindi than IndicTrans and vice versa
print(len(hi_result[hi_result.best_trans == hi_result.bart_translation])- 20)
print(len(hi_result[hi_result.best_trans == hi_result.indictrans_translation])- 20)

855
125


In [23]:
# Find the number of times that MBart produced a better translation for Tamil than IndicTrans and vice versa
print(len(ta_result[ta_result.best_trans == ta_result.bart_translation])- 20)
print(len(ta_result[ta_result.best_trans == ta_result.indictrans_translation])- 20)

811
154


In [24]:
# Find the number of times that MBart produced a better translation for Malayalam than IndicTrans and vice versa
print(len(ml_result[ml_result.best_trans == ml_result.bart_translation])- 20)
print(len(ml_result[ml_result.best_trans == ml_result.indictrans_translation])- 20)

619
341


In [25]:
# Save the target translations and the best translations for each language to a separate txt file
np.savetxt(D+"/predicted_text/siamese_evaluations_of_translations/hi_target.txt", hi_result.target_sentence.values, fmt='%s', newline='\n')
np.savetxt(D+"/predicted_text/siamese_evaluations_of_translations/hi_best.txt", hi_result.best_trans.values, fmt='%s', newline='\n')
np.savetxt(D+"/predicted_text/siamese_evaluations_of_translations/ta_target.txt", ta_result.target_sentence.values, fmt='%s', newline='\n')
np.savetxt(D+"/predicted_text/siamese_evaluations_of_translations/ta_best.txt", ta_result.best_trans.values, fmt='%s', newline='\n')
np.savetxt(D+"/predicted_text/siamese_evaluations_of_translations/ml_target.txt", ml_result.target_sentence.values, fmt='%s', newline='\n')
np.savetxt(D+"/predicted_text/siamese_evaluations_of_translations/ml_best.txt", ml_result.best_trans.values, fmt='%s', newline='\n')

In [26]:
# cd into the directory where the files are saved then compute Sacre Bleu scores
%cd /content/drive/MyDrive/W266_Project_Data/pmi_data/predicted_text/siamese_evaluations_of_translations/

/content/drive/MyDrive/W266_Project_Data/pmi_data/predicted_text/siamese_evaluations_of_translations


In [27]:
# Check for the output txt files
%ls

aryan_test.csv
bart_siamese_paraphrase_eval.csv
drav_test.csv
drav_test.gsheet
hi_best.txt
hi_target.txt
indictrans_siamese_paraphrase_eval.csv
ml_best.txt
ml_target.txt
paraphrase_test_eval_and_errors.csv
ta_best.txt
ta_target.txt
three_dense_paraphrase_test_eval_and_errors.csv
three_dense_w_punjabi_paraphrase_test_eval_and_errors.csv
three_dense_w_punjabi_paraphrase_test_eval_and_errors.gsheet
two_dense_w_punjabi_paraphrase_test_eval_and_errors.csv


In [28]:
!pip install sacrebleu -q

     |████████████████████████████████| 90 kB 4.9 MB/s 


In [29]:
# Get the Hindi Sacre Bleu scores
!sacrebleu hi_best.txt -i hi_target.txt --tokenize intl

{
 "name": "BLEU",
 "score": 68.2,
 "signature": "nrefs:1|case:mixed|eff:no|tok:intl|smooth:exp|version:2.0.0",
 "verbose_score": "85.5/73.2/63.1/54.8 (BP = 1.000 ratio = 1.019 hyp_len = 19557 ref_len = 19184)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "intl",
 "smooth": "exp",
 "version": "2.0.0"
}


In [30]:
# Get the Tamil translation Sacre Bleu scores
!sacrebleu ta_best.txt -i ta_target.txt --tokenize intl

{
 "name": "BLEU",
 "score": 44.5,
 "signature": "nrefs:1|case:mixed|eff:no|tok:intl|smooth:exp|version:2.0.0",
 "verbose_score": "69.7/50.1/38.0/29.4 (BP = 1.000 ratio = 1.065 hyp_len = 14425 ref_len = 13541)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "intl",
 "smooth": "exp",
 "version": "2.0.0"
}


In [31]:
# Get the Malayalam translation Sacre Bleu scores
!sacrebleu ml_best.txt -i ml_target.txt --tokenize intl

{
 "name": "BLEU",
 "score": 14.6,
 "signature": "nrefs:1|case:mixed|eff:no|tok:intl|smooth:exp|version:2.0.0",
 "verbose_score": "45.7/20.1/10.4/5.7 (BP = 0.959 ratio = 0.960 hyp_len = 11929 ref_len = 12425)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "intl",
 "smooth": "exp",
 "version": "2.0.0"
}
